In [1]:
import os
import dask.array as da
import networkx as nx
import zarr
from epynet import Network

import sys
sys.path.insert(0, os.path.join('..', 'utils'))
from graph_utils import get_nx_graph

In [2]:
wds_id = 'richmond'

In [3]:
path_to_data = os.path.join('..', 'data', 'db_'+wds_id+'_doe_pumpfed_1')
path_to_wds = os.path.join('..', 'water_networks', wds_id+'.inp')

# Loading data

In [4]:
wds = Network(path_to_wds)
G_unweighted = get_nx_graph(wds, mode='binary')
L_unweighted = da.array(nx.linalg.laplacianmatrix.laplacian_matrix(G_unweighted).todense())
G_weighted = get_nx_graph(wds, mode='weighted')
L_weighted = da.array(nx.linalg.laplacianmatrix.laplacian_matrix(G_weighted).todense())

store = zarr.open(path_to_data, mode='r')
X = da.from_zarr(store['trn/junc_heads']).T

In [5]:
L_unweighted

dask.array<array, shape=(865, 865), dtype=float64, chunksize=(865, 865), chunktype=numpy.matrix>

In [6]:
L_weighted

dask.array<array, shape=(865, 865), dtype=float64, chunksize=(865, 865), chunktype=numpy.matrix>

In [7]:
X

dask.array<transpose, shape=(865, 11998), dtype=float32, chunksize=(865, 11998), chunktype=numpy.ndarray>

# Smoothness

In [8]:
smoothness_unweighted = da.dot(X.T, da.dot(L_unweighted, X)).trace()
smoothness_weighted = da.dot(X.T, da.dot(L_weighted, X)).trace()

In [9]:
print(smoothness_unweighted.compute())
print(smoothness_weighted.compute())

11373596593.886005
757848140.257787
